In [ ]:
import numpy as np
from source.Files_operating import read_sht_data
from source.Peaks_processing import *

import time
import random

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib.colors import CSS4_COLORS as COLORS

colors = ['gold', 'brown', 'black', 'seagreen', 'skyblue', 'cyan', 'yellow', 'violet', 'royalblue', 'sandybrown', 'grey', 'gray', 'indigo', 'rosybrown', 'darkviolet', 'coral', 'pink', 'magenta', 'red', 'springgreen', 'darkblue', 'silver', 'seashell', 'green', 'navy', 'purple', 'sienna', 'chocolate', 'orange', 'blue']

In [ ]:
class Point: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y 
        
def get_area(x, y):
    n = len(x)
    area = 0
    for i in range(n):
        area += (x[i] * y[(i + 1) % n] - x[(i + 1) % n] * y[i])
    
    return 0.5 * area

def get_conv(x, y):
    convex = 0
    for i in range(len(x) - 2):
        v1 = np.array([x[i + 1] - x[i], y[i + 1] - y[i]])
        v2 = np.array([x[i + 2] - x[i + 1], y[i + 2] - y[i + 1]])
    
        cross_product = np.cross(v1, v2)
    
        if i == 0:
            sign = np.sign(cross_product)
        convex += sign * cross_product
    return convex

def get_intersections(x, y):
    """
    Подсчитывает количество самопересечений кривой, заданной массивами координат x и y.

    Args:
        x: Массив координат x.
        y: Массив координат y.

    Returns:
        Количество самопересечений.
    """
    step_out = 50
    intersections = []

    for i in range(len(x) - 1):
        for j in range(i - step_out):
            # Проверка пересечения отрезков (i, i + 1) и (j, j + 1)
            if doIntersect(Point(x[j], y[j]), Point(x[j + 1], y[j + 1]), Point(x[i], y[i]), Point(x[i + 1], y[i + 1])):
                intersections.append([j,i])

    return intersections
  
# Given three collinear points p, q, r, the function checks if  
# point q lies on line segment 'pr'  
def onSegment(p, q, r): 
    if ( (q.x <= max(p.x, r.x)) and (q.x >= min(p.x, r.x)) and 
           (q.y <= max(p.y, r.y)) and (q.y >= min(p.y, r.y))): 
        return True
    return False
  
def orientation(p, q, r): 
    # to find the orientation of an ordered triplet (p,q,r) 
    # function returns the following values: 
    # 0 : Collinear points 
    # 1 : Clockwise points 
    # 2 : Counterclockwise 
      
    # See https://www.geeksforgeeks.org/orientation-3-ordered-points/amp/  
    # for details of below formula.  
      
    val = (float(q.y - p.y) * (r.x - q.x)) - (float(q.x - p.x) * (r.y - q.y)) 
    if (val > 0): 
          
        # Clockwise orientation 
        return 1
    elif (val < 0): 
          
        # Counterclockwise orientation 
        return 2
    else: 
          
        # Collinear orientation 
        return 0
  
# The main function that returns true if  
# the line segment 'p1q1' and 'p2q2' intersect. 
def doIntersect(p1,q1,p2,q2): 
      
    # Find the 4 orientations required for  
    # the general and special cases 
    o1 = orientation(p1, q1, p2) 
    o2 = orientation(p1, q1, q2) 
    o3 = orientation(p2, q2, p1) 
    o4 = orientation(p2, q2, q1) 
  
    # General case 
    if ((o1 != o2) and (o3 != o4)): 
        return True
  
    # Special Cases 
  
    # p1 , q1 and p2 are collinear and p2 lies on segment p1q1 
    if ((o1 == 0) and onSegment(p1, p2, q1)): 
        return True
  
    # p1 , q1 and q2 are collinear and q2 lies on segment p1q1 
    if ((o2 == 0) and onSegment(p1, q2, q1)): 
        return True
  
    # p2 , q2 and p1 are collinear and p1 lies on segment p2q2 
    if ((o3 == 0) and onSegment(p2, p1, q2)): 
        return True
  
    # p2 , q2 and q1 are collinear and q1 lies on segment p2q2 
    if ((o4 == 0) and onSegment(p2, q1, q2)): 
        return True
  
    # If none of the cases 
    return False

In [ ]:
F_ID = 44184
proj_path = "C:/Users/f.belous/Work/Projects/Plasma_analysis"
# D:/Edu/Lab/Projects/Plasma_analysis | C:/Users/f.belous/Work/Projects/Plasma_analysis
sht_dir_path = proj_path + "/data/sht/all/"

dbs_dir_path = proj_path + "/data/dbs/sht/"
dbs_df = read_sht_data(f'Dref{F_ID}', dbs_dir_path, data_name="ch1")
channels = [1, 2, 3, 4, 5, 6]
for i in channels[1:]:
    dbs_df[f"ch{i}"] = read_sht_data(f'Dref{F_ID}', dbs_dir_path, data_name=f"ch{i}").ch1

In [ ]:
for ind in range(1, len(channels) + 1, 2):
    w = 0.8
    smooth_lenght = 500
    i_data = dbs_df[f"ch{ind}"].to_numpy() # normalise_series() filt_signal( , 5, w)  smooth_steklov( , smooth_lenght, smooth_lenght / 2)
    i_data /= np.linalg.norm(i_data)
    q_data = dbs_df[f"ch{ind + 1}"].to_numpy()
    q_data /= np.linalg.norm(q_data)

    c_data = filt_signal(i_data, 5, w) + filt_signal(q_data, 5, w)*1j

    fi_data = np.angle(c_data)

    dbs_df[f"ch{ind}_A"] = smooth_steklov(np.abs(c_data), smooth_lenght, 0.85)
    dbs_df[f"ch{ind}_dfi"] = smooth_steklov(np.concatenate([np.abs(np.diff(fi_data)), [0]]), smooth_lenght, 0.5)  # np.concatenate([np.diff(), [0]]) smooth_gauss(np.concatenate([filt_signal(np.diff(np.angle(c_data)), 5, w), [0]]), 300)

dbs_df.describe()

In [ ]:
points_df = pd.read_csv(f"data/df/stats/{F_ID}_points_dataset.csv")
print(points_df)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

time_points = points_df.timepoint.to_numpy()
marks = points_df.mark.to_numpy()
frequencies = points_df["fr, kHz"].to_numpy()

base_ch = 1

points_df["pred_mark-bypass"] = "eho"
points_df["pred_mark-fr"] = "-"
total = len(time_points)
t_delm, t_lco = 0, 0
f_delm, f_lco = 0, 0

print("Total samples:", total)
print("--------")
# print("|", end="")
start_time = time.time()
for p_i in range(total):
    if frequencies[p_i] > 7:
        points_df.loc[p_i, "pred_mark-fr"] = "eho"
    elif frequencies[p_i] > 3.0:
        points_df.loc[p_i, "pred_mark-fr"] = "lco"
    else:
        points_df.loc[p_i, "pred_mark-fr"] = "delm"

    # if p_i % 10 == 0:
    #     print(".", end="")
    
    if marks[p_i] == "eho":
        continue
    
    p_bypass = 1
    bypasses = []
    for ch_i in channels[::2]:
        step_out = [100, 50]
        time_mask = np.array((dbs_df.t * 1e6 >= time_points[p_i] - step_out[0]) & (dbs_df.t * 1e6 <= time_points[p_i] + step_out[1]))
        a = dbs_df[f"ch{base_ch}_A"][time_mask].to_numpy()
        dfi = dbs_df[f"ch{ch_i}_dfi"][time_mask].to_numpy()
        
        edges = [0, -1]
        
        intersections = get_intersections(a, dfi)  # sorted(get_intersections(a, dfi), key=lambda x: x[1] - x[0], reverse=True) 
        if len(intersections) > 0:
            edges = intersections[0]
        p_bypass *= np.sign(get_area(a[edges[0]:edges[1]], dfi[edges[0]:edges[1]]))
        bypasses.append(p_bypass)

    if p_bypass > 0:  # means - dELM
        points_df.loc[p_i, "pred_mark-bypass"] = "delm"
        if marks[p_i] == "delm":
            t_delm += 1
        else:
            f_delm +=1
    else:  # means - LCO
        points_df.loc[p_i, "pred_mark-bypass"] = "lco"
        if marks[p_i] == "lco":
            t_lco += 1
        else:
            f_lco +=1
            
    print(f"{p_i + 1}/{total}: {marks[p_i]} | {points_df.loc[p_i, 'pred_mark-bypass']} - {p_bypass} - {bypasses}")

print(f"Took: {time.time() - start_time:.3f} s")
print("--------")
print("\nFrequency:")
print("--------")
print(classification_report(points_df.mark.to_numpy(), points_df["pred_mark-fr"].to_numpy(), labels=["eho", "lco", "delm"]))

print("Confusion matrix")
cm = confusion_matrix(points_df.mark.to_numpy(), points_df["pred_mark-fr"].to_numpy(), labels=["eho", "lco", "delm"])
# matrix = cm.ravel()  # tn, fp, fn, tp
print(cm)
print("--------")

print("\nBypass:")
print("--------")
print(classification_report(points_df.mark.to_numpy(), points_df["pred_mark-bypass"].to_numpy(), labels=["eho", "lco", "delm"]))

print("Confusion matrix")
cm = confusion_matrix(points_df.mark.to_numpy(), points_df["pred_mark-bypass"].to_numpy(), labels=["eho", "lco", "delm"])
# matrix = cm.ravel()  # tn, fp, fn, tp
print(cm)

print("\n--------")
print(points_df)

In [ ]:
points_df.to_csv(f"data/df/stats/{F_ID}_points_dataset.csv", index=False)

---
---

In [ ]:
time_points = points_df.timepoint.to_numpy()
marks = points_df.mark.to_numpy()
frequencies = points_df["fr, kHz"].to_numpy()

comm = input("Input point indexes:\n")

base_ch = 1
while len(comm) > 0:
    diagram_points_ind = list(map(int, comm.strip().split()))
    diagram_points = time_points[diagram_points_ind]

    instability_bypass = np.ones(len(diagram_points))
    bypasses = []

    fig, axs = plt.subplots(ncols=len(channels[::2]), gridspec_kw={'hspace': 0.1})  # , sharex=True
    fig.set_figwidth(18)
    fig.set_figheight(9)
    
    fig.suptitle(f"$\partial\phi$(A) diagrams #{F_ID}")
    for ax_i, ch_i in enumerate(channels[::2]):
        axs[ax_i].set_title(f"Ch {ch_i}-{ch_i + 1}", fontsize="medium")
        # print(f"Areas for ch {ch_i}-{ch_i + 1}:")
        for p_i in range(len(diagram_points)):
            if marks[diagram_points_ind[p_i]] == "delm":
                step_out = [100, 50]
            elif marks[diagram_points_ind[p_i]] == "lco":
                step_out = [100, 50]
            elif marks[diagram_points_ind[p_i]] == "eho":
                step_out = [45, 40]
            else:
                step_out = [100, 50]
            time_mask = np.array((dbs_df.t * 1e6 >= diagram_points[p_i] - step_out[0]) & (dbs_df.t * 1e6 <= diagram_points[p_i] + step_out[1]))
            peak_inds = np.argwhere(np.array(abs(dbs_df.t * 1e6 - diagram_points[p_i]) < 1))
            # print(peak_inds, np.argwhere(time_mask))
            # t = dbs_df.t[time_mask].to_numpy()
            a = dbs_df[f"ch{base_ch}_A"][time_mask].to_numpy()
            dfi = dbs_df[f"ch{ch_i}_dfi"][time_mask].to_numpy()
            peak_ind = peak_inds[0] - np.argwhere(time_mask)[0]
            # print(dbs_df.t.to_numpy()[time_mask][9*40], dbs_df.t.to_numpy()[time_mask][16*40])
            
            edges = [0, -1]
            
            intersections = get_intersections(a, dfi)  # sorted(get_intersections(a, dfi), key=lambda x: x[1] - x[0], reverse=True)
            if len(intersections) > 0:  #  and intersections[0][1] - intersections[0][0] > 400
                # print(intersections[0][1] - intersections[0][0])
                edges = intersections[0]
                axs[ax_i].scatter(a[np.array(intersections)[:, 0]], dfi[np.array(intersections)[:, 0]], color="k", s=10, alpha=0.8)
                axs[ax_i].scatter(a[np.array(intersections)[:, 1]], dfi[np.array(intersections)[:, 1]], color="r", s=10, alpha=0.8)
            bypass = np.sign(get_area(a[edges[0]:edges[1]], dfi[edges[0]:edges[1]]))
            instability_bypass[p_i] *= bypass
            
            axs[ax_i].plot(a, dfi,
                           label=f"{diagram_points[p_i]/1e3} ms - {frequencies[diagram_points_ind[p_i]]:.1f} kHz - {'left' if bypass > 0 else 'right'} - {marks[diagram_points_ind[p_i]]}", alpha=0.8)
            axs[ax_i].scatter(a[0], dfi[0], color="k")
            axs[ax_i].scatter(a[-1], dfi[-1], color="r")
            axs[ax_i].scatter(a[peak_ind], dfi[peak_ind], color="violet", s=10)
            # axs[ax_i].scatter(a[::40], dfi[::40], color="b", s=5)

            # print(f"{p_i + 1}/{len(diagram_points)}: {get_area(a[edges[0]:edges[1]], dfi[edges[0]:edges[1]]) * 1e5:.1f} | {intersections} (len: {len(a)})")
        axs[ax_i].set_xlabel("A")
        axs[ax_i].set_ylabel("$\partial\phi$")
        axs[ax_i].grid(which='major', color='#DDDDDD', linewidth=0.9)
        axs[ax_i].grid(which='minor', color='#DDDDDD', linestyle=':', linewidth=0.7)
        axs[ax_i].minorticks_on()
        axs[ax_i].xaxis.set_minor_locator(AutoMinorLocator(10))
        axs[ax_i].legend(loc='upper left')
    plt.show()

    print("Diagram bypass prognose:")
    for p_i in range(instability_bypass.shape[0]):
        print(f"{p_i + 1}/{len(diagram_points)}: {'dELM' if instability_bypass[p_i] > 0 else 'LCO'}")
    print("\n")
    
    comm = input("Input point indexes:\n")

# 4 8 28 35 69 74 90
# 4 8 28 58 87 90
# 195.763 195.833: 9 - 16 * 40